In [20]:
import sys
sys.path.insert(0, "..")
import pictureweb.distributed.sharded_matrix as sharded_matrix
import pictureweb.distributed.distributed as distributed 
import boto3
import io
import numpy as np

In [27]:
def block_idxs_to_idxs(block_idxs, block_size, n):
    idxs = []
    for block in block_idxs:
        sidx = block_size*block
        eidx = min((block_size)*(block+1), n)
        idxs.extend(list(range(sidx, eidx)))
    return idxs

In [21]:
XXT_sharded = sharded_matrix.ShardedSymmetricMatrix("XXT(coatesng_16384_10_1_100_49_0.6311_0(imagenet_train_raw_uint8))", bucket="picturewebsolve")
X = sharded_matrix.ShardedMatrix("coatesng_16384_10_1_100_49_0.6311_0(imagenet_train_raw_uint8)", bucket="picturewebhyperband")

In [32]:
bloc_idx = 116
idxs = block_idxs_to_idxs([bidx], 4096, X.shape[0])

In [33]:
client = boto3.client('s3')
resp = client.get_object(Key="scrambled_train_labels.npy", Bucket="picturewebsolve")
bio = io.BytesIO(resp["Body"].read())
y_train = np.load(bio).astype('int')
y_train_enc = np.eye(1000)[y_train]

In [34]:
y_block = y_train_enc[idxs, :]

In [ ]:
X_block = distributed.get_column_block(X, column_blocks=[blo])